In [9]:
import os
import pandas as pd

# Función para explorar el tamaño de la base de datos y enlistar columnas
def explorar_dataframe(archivo):
    df = pd.read_parquet(archivo)  # Cargar el archivo parquet
    print(f"Explorando el archivo: {archivo}")
    print(f"Tamaño del DataFrame: {df.shape}")  # Muestra el tamaño (filas, columnas)
    print("Columnas del DataFrame:", df.columns.tolist())  # Lista de nombres de columnas
    print("\n")

# Ruta de la carpeta donde se encuentran los archivos parquet
folder_path = '/home/nicolas/despliegue_analytica/files_parquet/'

# Listar archivos parquet en el directorio
archivos_en_directorio = os.listdir(folder_path)

# Filtrar solo los archivos parquet
archivos_parquet = [archivo for archivo in archivos_en_directorio if archivo.endswith('.parquet')]

# Explorar cada archivo
for archivo in archivos_parquet:
    explorar_dataframe(os.path.join(folder_path, archivo))  # Combina la ruta de la carpeta con el nombre del archivo


Explorando el archivo: /home/nicolas/despliegue_analytica/files_parquet/olist_order_reviews_dataset.parquet
Tamaño del DataFrame: (104162, 7)
Columnas del DataFrame: ['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp']


Explorando el archivo: /home/nicolas/despliegue_analytica/files_parquet/product_category_name_translation.parquet
Tamaño del DataFrame: (71, 2)
Columnas del DataFrame: ['product_category_name', 'product_category_name_english']


Explorando el archivo: /home/nicolas/despliegue_analytica/files_parquet/log.parquet
Tamaño del DataFrame: (1312162, 13)
Columnas del DataFrame: ['cep_id', 'cep', 'tipo', 'nome_logradouro', 'logradouro', 'bairro_id', 'cidade_id', 'estado', 'complemento', 'latitude', 'longitude', 'cep_ativo', 'cep_ibge']


Explorando el archivo: /home/nicolas/despliegue_analytica/files_parquet/olist_order_payments_dataset.parquet
Tamaño del DataFrame: (103886, 5)
Columnas de

In [2]:
import os
import pandas as pd

# Lista de posibles columnas clave para joins (identificadores comunes)
posibles_llaves = ['order_id', 'customer_id', 'product_id', 'seller_id', 'cep', 'cep_prefix']

# Función para explorar el tamaño de la base de datos y enlistar columnas, indicando posibles llaves para join
def explorar_dataframe(archivo):
    df = pd.read_parquet(archivo)  # Cargar el archivo parquet
    columnas = df.columns.tolist()  # Lista de columnas
    llaves_encontradas = [col for col in columnas if col in posibles_llaves]  # Identificar llaves para join
    
    print(f"Explorando el archivo: {archivo}")
    print(f"Tamaño del DataFrame: {df.shape}")  # Muestra el tamaño (filas, columnas)
    print("Columnas del DataFrame:", columnas)  # Lista de nombres de columnas
    
    if llaves_encontradas:
        print(f"Posibles llaves para JOIN: {llaves_encontradas}")  # Muestra llaves si se encuentran
    else:
        print("No se encontraron posibles llaves para JOIN.")
    print("\n")

# Ruta de la carpeta donde se encuentran los archivos parquet
folder_path = '/home/nicolas/despliegue_analytica/files_parquet/'

# Listar archivos parquet en el directorio
archivos_en_directorio = os.listdir(folder_path)

# Filtrar solo los archivos parquet
archivos_parquet = [archivo for archivo in archivos_en_directorio if archivo.endswith('.parquet')]

# Explorar cada archivo
for archivo in archivos_parquet:
    explorar_dataframe(os.path.join(folder_path, archivo))  # Combina la ruta de la carpeta con el nombre del archivo


Explorando el archivo: /home/nicolas/despliegue_analytica/files_parquet/olist_order_reviews_dataset.parquet
Tamaño del DataFrame: (104162, 7)
Columnas del DataFrame: ['review_id', 'order_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp']
Posibles llaves para JOIN: ['order_id']


Explorando el archivo: /home/nicolas/despliegue_analytica/files_parquet/product_category_name_translation.parquet
Tamaño del DataFrame: (71, 2)
Columnas del DataFrame: ['product_category_name', 'product_category_name_english']
No se encontraron posibles llaves para JOIN.


Explorando el archivo: /home/nicolas/despliegue_analytica/files_parquet/log.parquet
Tamaño del DataFrame: (1312162, 13)
Columnas del DataFrame: ['cep_id', 'cep', 'tipo', 'nome_logradouro', 'logradouro', 'bairro_id', 'cidade_id', 'estado', 'complemento', 'latitude', 'longitude', 'cep_ativo', 'cep_ibge']
Posibles llaves para JOIN: ['cep']


Explorando el archivo: /home/nicol

In [17]:
import dask.dataframe as dd

# Ruta de la carpeta donde se encuentran los archivos parquet
folder_path = '/home/nicolas/despliegue_analytica/files_parquet/'

# Cargar los archivos parquet como Dask DataFrames
df_order_reviews = dd.read_parquet(folder_path + 'olist_order_reviews_dataset.parquet')
df_payments = dd.read_parquet(folder_path + 'olist_order_payments_dataset.parquet')
df_order_items = dd.read_parquet(folder_path + 'olist_order_items_dataset.parquet')
df_customers = dd.read_parquet(folder_path + 'olist_customers_dataset.parquet')
df_products = dd.read_parquet(folder_path + 'olist_products_dataset.parquet')
df_sellers = dd.read_parquet(folder_path + 'olist_sellers_dataset.parquet')
df_orders = dd.read_parquet(folder_path + 'olist_orders_dataset.parquet')
df_log = dd.read_parquet(folder_path + 'log.parquet')

# Convertir las columnas a string donde sea necesario
df_log['cep'] = df_log['cep'].astype(str)

# Realizar los joins progresivamente usando Dask
# 1. Unir orders con customers por 'customer_id'
df = dd.merge(df_orders, df_customers, on='customer_id', how='left')

# 2. Unir order_reviews con orders por 'order_id'
df = dd.merge(df, df_order_reviews, on='order_id', how='left')

# 3. Unir payments con orders por 'order_id'
df = dd.merge(df, df_payments, on='order_id', how='left')

# 4. Unir order_items con orders por 'order_id'
df = dd.merge(df, df_order_items, on='order_id', how='left')

# 5. Unir products con order_items por 'product_id'
df = dd.merge(df, df_products, on='product_id', how='left')

# 6. Unir sellers con order_items por 'seller_id'
df = dd.merge(df, df_sellers, on='seller_id', how='left')

# 7. Unir log (llave: cep) con el DataFrame resultante
df = dd.merge(df, df_log[['cep', 'logradouro', 'bairro_id', 'cidade_id', 'estado']], 
              left_on='customer_zip_code_prefix', right_on='cep', how='left')

# Guardar el resultado en un nuevo archivo parquet
output_path = '/home/nicolas/despliegue_analytica/files_parquet/olist_dataset_combinado.parquet'
df.to_parquet(output_path, write_index=False)

print("Join completo y archivo guardado en:", output_path)



Join completo y archivo guardado en: /home/nicolas/despliegue_analytica/files_parquet/olist_dataset_combinado.parquet


In [37]:
import dask.dataframe as dd

# Cargar el archivo parquet como Dask DataFrame
folder_path = '/home/nicolas/despliegue_analytica/files_parquet/'
df_combined = dd.read_parquet(folder_path + 'olist_dataset_combinado.parquet')

# Tamaño del DataFrame
num_rows, num_cols = df_combined.shape
print("Número de filas:", num_rows.compute())  # Computa el número de filas
print("Número de columnas:", num_cols)  # Ya es un entero

# Estadísticas descriptivas
stats = df_combined.describe()
print("Estadísticas descriptivas:\n", stats.compute())  # Computa las estadísticas y las muestra

# Para ver las primeras filas del DataFrame
print("Primeras filas del DataFrame:\n", df_combined.head())  # Muestra las primeras filas

# Conteo de valores nulos por columna
null_counts = df_combined.isnull().sum().compute()
print("Conteo de valores nulos por columna:\n", null_counts)

# Tipos de variables
dtypes = df_combined.dtypes  # Obtiene los tipos de variables
print("Tipos de variables:\n", dtypes)  # Muestra los tipos de variables (no necesita compute)



Número de filas: 119143
Número de columnas: 44
Estadísticas descriptivas:
             order_purchase_timestamp           order_approved_at  \
count                         119143                      118966   
min              2016-09-05 00:15:19         2016-09-15 15:16:38   
25%    2017-09-17 20:05:23.750000128  2017-09-19 02:41:29.500000   
50%              2018-01-24 23:06:24         2018-01-25 11:05:25   
75%              2018-05-07 23:48:25         2018-05-08 07:49:49   
max              2018-10-17 20:30:18         2018-09-03 20:40:06   
mean                            <NA>                        <NA>   
std                             <NA>                        <NA>   

      order_delivered_carrier_date order_delivered_customer_date  \
count                       117057                        115722   
min            2016-10-08 13:34:01           2016-10-11 16:46:32   
25%            2017-09-21 22:17:58           2017-10-03 21:53:05   
50%            2018-01-29 23:52:38      